# Build multi-1D models with maps

In [ ]:
import os
import multiprocessing
from pathlib import Path
import sys

sys.path.append("../")
import numpy as np
import temperer

In [ ]:
n_cpu = multiprocessing.cpu_count()
print("Available CPUs",n_cpu)

In [ ]:
os.getcwd()

In [ ]:
maps_dir = Path("../data/mapA")

Instantiate a model

In [ ]:
model = temperer.Model()

In [ ]:
inputs = model.builder.input_horizons_template

In [ ]:
#Add surface grids to the table. You can use other method as well
inputs.loc[0]=[0,"0.gri",None,"Onlap"]
inputs.loc[1]=[66,"66.gri",None,"Onlap"]
inputs.loc[2]=[100,"100.gri",None,"Onlap"]
inputs.loc[3]=[163,"163.gri",None,"Erosive"]
inputs.loc[4]=[168,"168.gri",None,"Erosive"]
inputs.loc[5]=[170,"170.gri",None,"Onlap"]
inputs.loc[6]=[182,"182.gri",None,"Erosive"]
model.builder.input_horizons=inputs
inputs

Make a grid of locations or specify X, Y extraction

In [ ]:
inc = 2000
model.builder.define_geometry(maps_dir/"0.gri",xinc=inc,yinc=inc,fformat="irap_binary")

In [ ]:
%%time
model.builder.extract_nodes(4,maps_dir)

In [ ]:
model.parameters.time_start

In [ ]:
from temperer.data import haq87
model.builder.set_eustatic_sea_level(haq87)

In [ ]:
for i in model.builder.iter_node():
    i.rift=[[182,175]]


Create an input data table for horizon and age extraction

In [ ]:
# simulate every 10 nodes
for index in model.builder.grid.indexing_arr:
    if (index[0] % 10 > 0):
        pass
    else:
        if isinstance(model.builder.nodes[index[0]][index[1]],bool) is False:
            model.builder.nodes[index[0]][index[1]] = False
    if (index[1] % 10 > 0):
        pass
    else:
        if isinstance(model.builder.nodes[index[0]][index[1]],bool) is False:
            model.builder.nodes[index[0]][index[1]] = False


In [ ]:
indexer_full_sim = [i.indexer for i in model.builder.iter_node() if i._full_simulation is True]

In [ ]:
model.builder.nodes[1][1]

In [ ]:
%%time
model.simulator.run(save=False,purge=True)

In [ ]:
model.builder.nodes[0][1]

In [ ]:
for i in model.builder.iter_node():
    if i is not False:
        print(i.result.heatflow(0))